# Brightway2 seminar
Chris Mutel ([PSI](https://www.psi.ch/)), Pascal Lesage ([CIRAIG](http://www.ciraig.org/en/))

## Day 1, afternoon  
Session on working with `Activity` and ` Exchange` proxies 

### Learning objectives  
  - Work with `Activity` and ` Exchange` proxies (copy, add, modify, delete)  

**Exercise**: Create a new activity for blow moulding in the USA that is based on a copy of the activity `'stretch blow moulding`. It is to have the following modifications:  
  - Its location is to be changed to "US" ==> changing an activity
  - Its source of electricity is to be changed to medium voltage electricity from the US grid ==> changing an exchange  
  - The waste heat elementary flow is to be deleted ==> Deleting an exchange  
  - The cooling water is to come from the "river" subcompartment ==> Deleting and adding an exchange

CHRIS: this may be fun, but probably would take too long.
  - The Pedigree matrix scores for the exchanges are to be modified  
  - The uncertainty is to be modified based on these modified Pedigree scores

**Step 1**: Import Brightway as bw, open the seminar project and make a copy of the ecoinvent v2.2 database.

In [1]:
import brightway2 as bw

In [2]:
bw.projects.set_current('bw2_seminar_2017')

In [3]:
if 'ecoinvent 2.2 copy' not in bw.databases:
    bw.Database('ecoinvent 2.2').copy('ecoinvent 2.2 copy')
else:
    print('A copy already exists')
eidb = bw.Database('ecoinvent 2.2 copy')

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:05:36


Title: Writing activities to SQLite3 database:
  Started: 03/30/2017 23:11:56
  Finished: 03/30/2017 23:17:32
  Total time elapsed: 00:05:36
  CPU %: 11.60
  Memory %: 6.18


**Step 2**: Create a copy of the activity `'offset printing, per kg printed paper' (kilogram, CH, None)` and assign it to a variable.  

In [4]:
bm_RER = [act for act in eidb if 'stretch blow moulding' in act['name']][0]
bm_RER

'stretch blow moulding' (kilogram, RER, ['plastics', 'processing'])

In [5]:
bm_US = bm_RER.copy()

Before we go on, let's get a sense of what the activity looks like and what exchanges are output into the activity.

In [6]:
bm_US.as_dict()

{'authors': [{'address': 'Lerchenfeldstrasse 5, 9014 St. Gallen',
   'company': 'EMPA',
   'country': 'CH',
   'email': 'empa@ecoinvent.org',
   'name': 'Roland Hischier'}],
 'categories': ['plastics', 'processing'],
 'code': '528f0f9235414af8acefda3176267acc',
 'comment': '1 kg of this process equals 0.978 kg of strech blow moulded plastics.\nThis process contains the auxillaries and energy demand for the mentioned convertion process of plastics. The converted amount of plastics is NOT included into the dataset.\nLocation:  information from different European and Swiss converting companies\nTechnology:  present technologies\nTime period:  time to which data refer\nProduction volume:  unknown\nSampling:  company data\nExtrapolations:  no extrapolation\nUncertainty:  none',
 'database': 'ecoinvent 2.2 copy',
 'filename': 'E:\\datasets\\01855.XML',
 'location': 'RER',
 'name': 'stretch blow moulding',
 'production amount': 1.0,
 'type': 'process',
 'unit': 'kilogram'}

In [ ]:
for exc in bm_US.exchanges():
    print(exc.input['name'], exc['amount'], exc['unit'])

**Step 3**: Just a check: make sure that the key of the activities are not the same

In [7]:
bm_RER.key == bm_US.key

False

**Step 4**: Change the US activity's location to "US"

In [8]:
bm_US['location']="US"
bm_US.save()

**Step 5**: Find the electricity exchange to be replaced in the US printing activity.

In [9]:
electricity_input_exchange = [exc for exc in bm_US.technosphere() if "electricity" in exc['name']][0]
electricity_input_exchange

Exchange: 2.55 kilowatt hour 'electricity, medium voltage, production UCTE, at grid' (kilowatt hour, UCTE, ['electricity', 'production mix']) to 'stretch blow moulding' (kilogram, US, ['plastics', 'processing'])>

**Step 6**: Find the activity that this exchange needs to be relinked to.

In [10]:
US_elect=[act for act in eidb if 'electricity, medium voltage' in act['name']
          and act['location']=='US'
         ][0]
US_elect

'electricity, medium voltage, at grid' (kilowatt hour, US, ['electricity', 'supply mix'])

**Step 7**: Relink the electricity input exchange to the US electricity:

In [11]:
electricity_input_exchange.input = US_elect

In [12]:
for exc in bm_US.technosphere():
    if "electricity" in exc.input['name']:
        print(exc.input['name'], exc.input['location']) 

electricity, medium voltage, production UCTE, at grid UCTE


**Step 8**: Delete the waste heat elementary flow:

In [13]:
waste_heat = [exc for exc in bm_US.biosphere() if "Heat, waste" in exc['name']][0]
waste_heat

Exchange: 9.17 megajoule 'Heat, waste' (megajoule, None, ('air', 'urban air close to ground')) to 'stretch blow moulding' (kilogram, US, ['plastics', 'processing'])>

In [14]:
waste_heat.delete()

**Step 9**: Create a new exchange for the water input from river and delete the old cooling water, unspecified

In [15]:
[ef for ef in bw.Database('biosphere3') if "river" in ef['name']]  # To determine how best to return my activity
river_water = [ef for ef in bw.Database('biosphere3') if "Water, river" in ef['name']][0]
river_water

'Water, river' (cubic meter, None, ('natural resource', 'in water'))

In [16]:
# Also return the existing cooling water flow to "steal" some information from it
old_cooling_water = [ef for ef in bm_US.biosphere() if "cooling" in ef['name']][0]
old_cooling_water

Exchange: 0.11 cubic meter 'Water, cooling, unspecified natural origin' (cubic meter, None, ('natural resource', 'in water')) to 'stretch blow moulding' (kilogram, US, ['plastics', 'processing'])>

In [17]:
old_cooling_water.as_dict()

{'amount': 0.11,
 'categories': ['natural resource', 'in water'],
 'comment': '(2,3,2,1,3,4); typical values, based on a European and a Swiss study',
 'input': ('biosphere3', 'fc1c42ce-a759-49fa-b987-f1ec5e503db1'),
 'loc': -2.2072749131897207,
 'name': 'Water, cooling, unspecified natural origin',
 'negative': False,
 'output': ('ecoinvent 2.2 copy', '528f0f9235414af8acefda3176267acc'),
 'scale': 0.13118213223374559,
 'type': 'biosphere',
 'uncertainty type': 2,
 'unit': 'cubic meter'}

In [18]:
new_water = bm_US.new_exchange(input=river_water.key,
                   amount=old_cooling_water['amount'],
                   unit=old_cooling_water['unit'],
                   categories=old_cooling_water['categories'],
                   type='biosphere')

In [19]:
new_water.save()

Then delete the old water input:

In [20]:
old_cooling_water.delete()

In [21]:
bm_US.save()

**Final result:** 

In [22]:
for exc in bm_US.exchanges():
    try:
        print(exc.input['name'], exc['amount'], exc['unit'], exc.input['location'])
    except:
        print(exc.input['name'], exc['amount'], exc['unit'])

lubricating oil, at plant 0.00196 kilogram RER
solid bleached board, SBB, at plant 0.0323 kilogram RER
electricity, medium voltage, production UCTE, at grid 2.55 kilowatt hour UCTE
packaging box production unit 1.43e-09 unit RER
disposal, plastics, mixture, 15.3% water, to municipal incineration 0.0215 kilogram CH
stretch blow moulding 1.0 kilogram US
Water, river 0.11 cubic meter
